# **Rete Neurale**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#Normal libraries

from pandas import read_csv
import pandas as pd
import numpy as np
import datetime

#Library to build a model using keras that is build on tensorflow
#(a library on how to handle tensor and their function)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

#Libraries about the learning process of the actual AI
from keras.callbacks import ModelCheckpoint
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

#Useful libraries, in these days I'll try to learn pyplot, probably could be way better
from matplotlib import pyplot as plt
import math

In [ ]:
#These Dataset are easier to handle, have little less information but are still usefull to start

df8 = read_csv("./DataSet_FBK/StChiara.csv",sep=",",header=0)
df_APPA = read_csv("./DataSet_APPA/APPA1.csv", sep = ",",header = 0)

df8.ts = pd.to_datetime(df8.ts)
df_APPA.Time = pd.to_datetime(df_APPA.Time)

print(df8)
print(df_APPA)

a1 = "mean_absolute_error"
a2 = "mean_squared_error"
aper = "mean_absolute_percentage_error"

In [ ]:
timezone_offset = datetime.timedelta(hours=2)
timezone = datetime.timezone(timezone_offset)

df_APPA.Time = df_APPA.Time.apply(lambda x: x.replace(tzinfo=timezone))
df_APPA.rename(columns = {"Time":"ts"}, inplace = True)

In [ ]:



print(df8.describe())

In [ ]:
df_tot = df8.set_index("ts").resample("1H").mean().reset_index().merge(df_APPA)

In [ ]:
change = {}
for i in range(1,9):
    change["S" +str(i)+"_R1"] = "T" + str(i)
    change["S" +str(i)+"_R2"] = "R" + str(i)
    change["S"+ str(i)+"_Voltage"] = "V"+str(i)


In [ ]:
#Just to convert them all to flaot and to cancel the missing data. Notice that by missing data in this
# case I am referring to actual missing data, where for example the push to the database process failed 
# we are not adressing the possibility of broken sensor or broken pump or time drift
df8 = df_tot
df8 = df8.rename(columns=change)
df8.ts = df8.ts.apply(lambda x: x.replace(tzinfo=None))
df8 = df8.set_index("ts")
df8 = df8.apply(pd.to_numeric, errors='coerce')
df8 = df8.dropna()
#You can print the correlaction table with all the data, notice that you can focus on 
# fixed interval to see if the table is
#different, and try to get why
corrcomplete = df8.corr()
corrcomplete.style.background_gradient(cmap='coolwarm')

In [ ]:
#This is really an important function as it can permit you to see the data in a more readable way. Notice that you can tweak
#all the dependence of the plot that I defined and used. The dataset that you see is already filtered as I added the provision
# df8["T" + str(i)] < 160, this is because if you have the resistance of the Heater over 200 Ohm the single sensor is broken.
#You can check this intuition just by taking out that provision and see what else you can notice, maybe taking about other problem
#about sensor signal and sensor replacements

import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(25, 30))
fig.suptitle('Gas')
outer = gridspec.GridSpec(4,2, wspace=0.2, hspace=0.2)
cl=df8.columns
annostr = ["R","T"]
gas=["PM10","Biossido di Azoto","Ozono","Biossido Zolfo"]
color=["blue","green"]
count = 0
for i in range(1,9):
    inner = gridspec.GridSpecFromSubplotSpec(2, 1,
                    subplot_spec=outer[i-1], wspace=0.1, hspace=0.2)

    for j in range(2):
        ax = plt.Subplot(fig, inner[j])
        ax.plot(df8.loc[df8["T" + str(i)] < 160][:].index, df8.loc[df8["T" + str(i)] < 160][:][annostr[j]+str(i)],'tab:'+color[j], label=annostr[j]+" "+str(i))
        ax.legend()
        fig.add_subplot(ax)
#fig.savefig('APPA1-2021-22.png', dpi=600, bbox_inches='tight')
fig.show()

# A Little detailed example on how to visualize data

In [ ]:
import plotly.express as px
fig = px.scatter(x=df8["1/1/2021":"30/06/2021"].R1, y=df8["1/1/2021":"30/06/2021"].R2, color = df8["1/1/2021":"30/06/2021"]["T"])
fig.show()

In [ ]:
#This is an example of how to perform some data visualization. Take note that plotly is probably way better to do the exact same


#For now I will just teach you the basic of to make plots with matplotlib, in general if you have question, ask me or ask google
#The second one knows more things usually 
#You are defining the "Canvas" or the space where you are drawing the plots
fig,(ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 7))
#Definition of the two plots, notice that you can add a third dimension through the use of the color of the points,
#Again plotly actually seems to do the job in a better way, you can tweak the variable to see if you can notice something
ax1.scatter(df8["07/01/2021":]["Biossido di Azoto"], df8["07/01/2021":].R2, c =  df8["07/01/2021":].T2,label='Gennaio-Giugno 2021')
ax2.scatter(df8["07/01/2021":]["Ozono"], df8["07/01/2021":].R2, c =  df8["07/01/2021":].T2,label='Gennaio-Giugno 2021')

#Legends, title and title axes of the two graph
ax2.set_title('Answer of LaFeO3 to NO2')
ax2.set_xlabel('NO2 Concentration')
ax2.set_ylabel('LaFeO3  Sensor signal')
ax1.set_title('Answer of LaFeO3 to O3')
ax1.set_xlabel('O3 Concentration')
ax1.set_ylabel('LaFeO3  Sensor signal')
#You can in general: set a limit to each axis, add text, save the figure, use a logaritmic scale either on the X or the Y, just try and see what works or not
#ax2.set_ylim((0,0.2*10**8))
#ax2.text(0.02, 0.86, "$\mathregular{R^2}$ = " + str(round(r_squared,3)) + "  RMS  = "  + str(round(rms,3))+ " ppb", transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.legend()
#plt.savefig('PM10allRPredict.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
fig,(ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(20, 7))
ax1.scatter(df8["07/01/2021":"12/31/2021"]["Biossido di Azoto"], df8["07/01/2021":"12/31/2021"].R2, c = df8["07/01/2021":"12/31/2021"].T2)
ax2.scatter(df8["07/01/2021":"12/31/2021"]["Ozono"], df8["07/01/2021":"12/31/2021"].R2, c = df8["07/01/2021":"12/31/2021"].T2)
ax2.set_title('Risposta SmFeO3 nel tempo')
ax2.set_xlabel('Concentrazione Biossido di Azoto')
ax2.set_ylabel('SmFeO3  Risposta')
#ax2.set_ylim((0,6400000)) 
#ax2.set_ylim((0,7500000)) TEMP
#ax2.set_ylim((0.075*10**8,0.16*10**8))
#ax2.text(0.02, 0.86, "$\mathregular{R^2}$ = " + str(round(r_squared,3)) + "  RMS  = "  + str(round(rms,3))+ " ppb", transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.legend()
#plt.savefig('PM10allRPredict.png', dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(8,5))
ax2 = fig.add_subplot()
ax = ax2.twinx()
ax2.plot(df8["07/01/2021":"07/10/2021"].index, df8["07/01/2021":"07/10/2021"].Ozono, label='First Month')
ax.plot(df8["07/01/2021":"07/10/2021"].index, df8["07/01/2021":"07/10/2021"].R5, c = "red", alpha = 0.2, label='March-July 2021')
ax2.set_title('Initial lag of ZnOR')
ax2.set_xlabel('Ozone Concentration')
ax2.set_ylabel('ZnOR Sensor Signal')
ax2.legend()
plt.show()

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(8,5))
ax2 = fig.add_subplot()
ax2.scatter(df8["01/15/2022":"02/15/2022"]["Biossido di Azoto"], df8["01/15/2022":"02/15/2022"].R7, label='Jan 2022')
ax2.scatter(df8["02/15/2022":"03/08/2022"]["Biossido di Azoto"], df8["02/15/2022":"03/08/2022"].R7, label='Feb - March 2022')
ax2.scatter(df8["03/08/2022":"04/03/2022"]["Biossido di Azoto"], df8["03/08/2022":"04/03/2022"].R7, label='March - April 2022')
ax2.set_title('SmFeO3')
ax2.set_xlabel('Nitrogen Dioxide Concentration')
ax2.set_ylabel('SmFeO3  Sensor Signal')
ax2.legend()
plt.show()

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(8,5))
ax2 = fig.add_subplot()
ax = ax2.twinx()
ax2.plot(df8["01/15/2022":"04/03/2022"].index, df8["01/15/2022":"04/03/2022"].R7)
ax.plot(df8["01/15/2022":"04/03/2022"].index, df8["01/15/2022":"04/03/2022"]["Biossido di Azoto"], c="r" )
ax2.set_title('Change in Sensor Answer')
ax2.set_ylabel('SmFeO3 Answer')
ax.set_ylabel('Concentration of Nitrogen Dioxide')
plt.show()

In [ ]:
#Here you can see a pump fail in good detail, can you find something similar in the full data, or other "funny" things_

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(8,5))
ax2 = fig.add_subplot()
ax2.plot(df8["03/05/2022":"03/08/2022"].index, df8["03/05/2022":"03/08/2022"].R4, label = "R4")
ax2.plot(df8["03/05/2022":"03/08/2022"].index, df8["03/05/2022":"03/08/2022"].R3, label = "R3")
ax2.plot(df8["03/05/2022":"03/08/2022"].index, df8["03/05/2022":"03/08/2022"].R8, label = "R8")
ax2.plot(df8["03/05/2022":"03/08/2022"].index, df8["03/05/2022":"03/08/2022"].R5, label = "R5")
ax2.plot(df8["03/05/2022":"03/08/2022"].index, df8["03/05/2022":"03/08/2022"].R6, label = "R6")
ax2.set_title('Example of the effect of a pump problem')
ax2.set_ylabel('Sensor Resistance [Ω]')
ax2.set_xlabel('Time')
ax2.legend()
plt.show()

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(8,5))
ax2 = fig.add_subplot()
ax = ax2.twinx()
ax2.plot(df8["03/15/2022":"04/02/2022"].index, df8["03/15/2022":"04/02/2022"].R7)
ax.plot(df8["03/15/2022":"04/02/2022"].index, df8["03/15/2022":"04/02/2022"]["Biossido di Azoto"], c="r", label = "Biossido d'Azoto" )
ax2.set_title('Detector SmFeO3 in time')
ax2.set_ylabel('SmFeO3 Sensor signal')
ax.set_ylabel('Concentration of Nitrogen Dioxide')
ax2.set_ylim((2.9*10**7,4.2*10**7))
ax.legend()
plt.show()

# Samario Ferro

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(8,5))
ax2 = fig.add_subplot()
ax2.scatter(df8["01/15/2022":"02/15/2022"]["Biossido di Azoto"], df8["01/15/2022":"02/15/2022"].R7, label='Gennaio 2022')
ax2.scatter(df8["02/15/2022":"03/08/2022"]["Biossido di Azoto"], df8["02/15/2022":"03/08/2022"].R7, label='Metà Febbraio - Metà Marzo 2022')
ax2.scatter(df8["03/08/2022":"04/03/2022"]["Biossido di Azoto"], df8["03/08/2022":"04/03/2022"].R7, label='Metà Marzo - Metà Aprile 2022')
ax2.scatter(df8["04/03/2022":]["Biossido di Azoto"], df8["04/03/2022":].R7, label='Post Cambio Pompa')
ax2.set_title('Detector SmFeO3 in time')
ax2.set_ylabel('SmFeO3 Sensor signal')
ax.set_ylabel('Concentration of Nitrogen Dioxide')
ax2.legend()
plt.show()

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
ax = ax2.twinx()
ax2.scatter(df8["01/15/2022":"02/15/2022"].index, df8["01/15/2022":"02/15/2022"].TEMP)
ax2.scatter(df8["02/15/2022":"03/08/2022"].index, df8["02/15/2022":"03/08/2022"].TEMP)
ax2.scatter(df8["03/08/2022":"04/03/2022"].index, df8["03/08/2022":"04/03/2022"].TEMP)
ax2.scatter(df8["04/03/2022":].index, df8["04/03/2022":].TEMP)

ax2.set_title('Strange Temperature drops, who knows what really happens')
ax2.set_xlabel('Time')
ax2.set_ylabel('Temperature')
plt.show()

In [ ]:
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
ax = ax2.twinx()
ax2.scatter(df8["01/15/2022":"02/15/2022"].index, df8["01/15/2022":"02/15/2022"].HUM, label='Jan 2022')
ax2.scatter(df8["02/15/2022":"03/08/2022"].index, df8["02/15/2022":"03/08/2022"].HUM, label='Feb - March 2022')
ax2.scatter(df8["03/08/2022":"04/03/2022"].index, df8["03/08/2022":"04/03/2022"].HUM, label='March - April 2022')
ax2.scatter(df8["04/03/2022":].index, df8["04/03/2022":].HUM, label='Post Cambio Pompa')

ax2.set_title('Humidity over time')
ax2.set_xlabel('Time')
ax2.set_ylabel('Humidity Sensor')
ax2.legend()
plt.show()

In [ ]:
corrcomplete1 = df8["02/15/2022":"03/09/2022"].corr()

corrcomplete1.style.background_gradient(cmap='coolwarm')

In [ ]:
corrcomplete2 = df10.corr()

corrcomplete2.style.background_gradient(cmap='coolwarm')

Real naive example of a machine learning process, name all the error that are made in it.

In [ ]:
#Real Machine Learning part, split the data and the object

X = df8.drop('Ozono', axis = 1)
X = X.drop('Biossido Zolfo', axis = 1)
X = X.drop('PM10', axis = 1)
X = X.drop('Biossido di Azoto', axis = 1)
X = X.drop('Ossido di Carbonio', axis = 1)

y = df8[['Ozono','Biossido Zolfo','PM10','Biossido di Azoto','Ossido di Carbonio']].copy()

INPUT_DIM = 18

In [ ]:
#I just called the test validation set, but I promis you that it will make someday sense
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 20)

In [ ]:
#Scale data, otherwise model will fail.
#Standardize features by removing the mean and scaling to unit variance
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)

scaler=StandardScaler()
scaler.fit(y_train)

y_train_s = scaler.transform(y_train)
y_val_s = pd.DataFrame(data = scaler.transform(y_val), columns = y_val.columns)

In [ ]:
# define the model
#Experiment with deeper and wider networks
model = Sequential()

# Input layer

model.add(Dense(128, kernel_initializer='normal',input_dim=INPUT_DIM, activation='selu'))
model.add(Dropout(0.2))


# The Hidden Layers :
model.add(Dense(256, kernel_initializer='normal',activation='selu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='normal',activation='selu'))
model.add(Dropout(0.2))

model.add(Dense(256, kernel_initializer='normal',activation='selu'))
model.add(Dropout(0.2))

#Output layer
model.add(Dense(5, kernel_initializer='normal',activation='linear'))

model.compile(loss=a2, optimizer='adam', metrics=[a2])
model.summary()

checkpoint_name = './tmp/checkpoint' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='min')
callbacks_list = [checkpoint]

history = model.fit(X_train_scaled, y_train_s, validation_split=0.2, epochs=300,callbacks=[checkpoint])
model.load_weights(checkpoint_name)
#wights_file = 'Weights-478--18738.19831.hdf5' # choose the best checkpoint 
# # load it
#model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

## Prediction on test

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['mean_squared_error']
val_acc = history.history['val_mean_squared_error']
plt.plot(epochs, acc, 'y', label='Training MAE')
plt.plot(epochs, val_acc, 'r', label='Validation MAE')
plt.title('Training and validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

############################################
#Predict on test data
predictions = model.predict(X_val_scaled)

predictions = scaler.inverse_transform(predictions)

y_val_ssaved = y_val_s
y_val_s = y_val
print("Predicted values are: ", predictions)
print("Real values are: ", y_val_s)

##############################################
#Plot and compare prediction and real value
t = np.arange(0, predictions.size, 1)
plt.scatter(predictions[:,0], y_val_s["Ozono"], label='Prediction')
plt.scatter(y_val_s["Ozono"], y_val_s["Ozono"], label='O3 value')
plt.title('Prediction O3')
plt.xlabel('Prediction')
plt.ylabel('Real Value')
plt.legend()
plt.savefig('Ozono.png', dpi=300, bbox_inches='tight')
plt.show()


t = np.arange(0, predictions.size, 1)
plt.scatter(predictions[:,1], y_val_s["PM10"], label='Prediction')
plt.scatter(y_val_s["PM10"], y_val_s["PM10"], label='PM10 value')
plt.title('Prediction PM10')
plt.xlabel('Prediction')
plt.ylabel('Real Value')
plt.legend()
plt.show()


t = np.arange(0, predictions.size, 1)
plt.scatter(predictions[:,2], y_val_s["Biossido Zolfo"], label='Prediction')
plt.scatter(y_val_s["Biossido Zolfo"], y_val_s["Biossido Zolfo"], label='Biossido Zolfo value')
plt.title('Prediction Biossido Zolfo')
plt.xlabel('Prediction')
plt.ylabel('Real Value')
plt.legend()
plt.show()


t = np.arange(0, predictions.size, 1)
plt.scatter(predictions[:,3], y_val_s["Biossido di Azoto"], label='Prediction')
plt.scatter(y_val_s["Biossido di Azoto"], y_val_s["Biossido di Azoto"], label='Biossido di Azoto value')
plt.title('Prediction Biossido di Azoto')
plt.xlabel('Prediction')
plt.ylabel('Real Value')
plt.legend()
plt.show()


t = np.arange(0, predictions.size, 1)
plt.scatter(predictions[:,4], y_val_s["Ossido di Carbonio"], label='Prediction')
plt.scatter(y_val_s["Ossido di Carbonio"], y_val_s["Ossido di Carbonio"], label='Ossido di Carbonio value')
plt.title('Prediction Ossido di Carbonio')
plt.xlabel('Prediction')
plt.ylabel('Real Value')
plt.legend()
plt.show()


In [ ]:
#From here on there is a lot a graph that I used to gain some information, I don't really want ot edit them until 3 AM, so I'm
#sorry if they are in english, you can just ignore them or try to take a look at them, I don't really know if they work anymore

supportografico = range(len(y_val_s) )
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico, y_val_s["Ossido di Carbonio"]-predictions[:,4], 'y', label='$\mathregular{O_3}$ value')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('Prediction $\mathregular{O_3}$')
plt.xlabel('Time')
plt.ylabel('Difference between calculated and measured $\mathregular{O_3}$')
plt.legend()
plt.show()

In [ ]:
supportografico = range(0,100)
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico, y_val_s["Ossido di Carbonio"][:100], 'y', label='CO value')
plt.plot(supportografico, predictions[:100,4], 'r', label='Prediction')
plt.title('Prediction CO')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
supportografico = range(0,100)
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico, y_val_s["PM10"][:100], 'y', label='PM10 value')
plt.plot(supportografico, predictions[:100,1], 'r', label='Prediction')
plt.title('Prediction PM10')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
supportografico = range(0,100)
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico, y_val_s["Biossido di Azoto"][:100], 'y', label='Biossido di Azoto value')
plt.plot(supportografico, predictions[:100,3], 'r', label='Prediction')
plt.title('Prediction Biossido di Azoto')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.show()

In [ ]:
df8.head()

In [ ]:

fig,ax = plt.subplots(figsize=(8,4))
ax.scatter(y["PM10"],y["Ossido di Carbonio"])
ax.set_xlabel("PM10",fontsize=14)
ax.set_ylabel("Ossido di Carbonio",color="red",fontsize=14)
plt.show()

support = range(len(y))
fig,ax = plt.subplots(figsize=(20,4))
ax.step(support,y["PM10"],label = "PM10" )
ax.set_xlabel("time",fontsize=14)
ax.set_ylabel("concentration [ppm]",color="red",fontsize=14)
ax2=ax.twinx()
ax2.step(support,y["Ossido di Carbonio"],color="green",label = "Ossido di Carbonio" )
ax2.set_ylabel("concentration [ppb]",color="red",fontsize=14)
ax.legend()
ax2.legend()

plt.show()

In [ ]:
support = range(len(df8))
fig,ax = plt.subplots(figsize=(20,4))
ax.step(support,y["Biossido di Azoto"],label = "Biossido di Azoto" )
ax.set_xlabel("time",fontsize=14)
ax.set_ylabel("concentration [ppm]",color="red",fontsize=14)
ax2=ax.twinx()
ax2.step(support,y["Ossido di Carbonio"],color="green",label = "Ossido di Carbonio" )
ax2.set_ylabel("concentration [ppb]",color="red",fontsize=14)
ax.legend()
ax2.legend()

plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))

ax.set_xlabel("Biossido di Azoto",fontsize=14)
ax.set_ylabel("Ossido di Carbonio",color="red",fontsize=14)

ax.scatter(y["Biossido di Azoto"],y["Ossido di Carbonio"])
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
ax.scatter(y["PM10"],y["Biossido di Azoto"])
ax.set_xlabel("PM10",fontsize=14)
ax.set_ylabel("Biossido di Azoto",color="red",fontsize=14)
plt.show()

support = range(len(y))
fig,ax = plt.subplots(figsize=(20,4))
ax.step(support,y["PM10"],label = "PM10" )
ax.set_xlabel("time",fontsize=14)
ax.set_ylabel("concentration [ppm]",color="red",fontsize=14)
ax2=ax.twinx()
ax2.step(support,y["Biossido di Azoto"],color="green",label = "Biossido di Azoto" )
ax2.set_ylabel("concentration [ppb]",color="red",fontsize=14)
ax.legend()
ax2.legend()

plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
ax.scatter(y["PM10"],y["Biossido di Azoto"])
ax.set_xlabel("PM10",fontsize=14)
ax.set_ylabel("Biossido di Azoto",color="red",fontsize=14)
plt.show()

support = range(len(y))
fig,ax = plt.subplots(figsize=(20,4))
ax.step(support,y["PM10"],label = "PM10" )
ax.set_xlabel("time",fontsize=14)
ax.set_ylabel("concentration [ppm]",color="red",fontsize=14)
ax2=ax.twinx()
ax2.step(support,y["Biossido di Azoto"],color="green",label = "Biossido di Azoto" )
ax2.set_ylabel("concentration [ppb]",color="red",fontsize=14)
ax.legend()
ax2.legend()

plt.show()


In [ ]:
fig,ax = plt.subplots(figsize=(8,4))
ax.scatter(y["PM10"],y["Biossido di Azoto"])
ax.set_xlabel("PM10",fontsize=14)
ax.set_ylabel("Ozono",color="red",fontsize=14)
plt.show()

support = range(len(y))
fig,ax = plt.subplots(figsize=(20,4))
ax.step(support,y["PM10"],label = "PM10" )
ax.set_xlabel("time",fontsize=14)
ax.set_ylabel("concentration [ppm]",color="red",fontsize=14)
ax2=ax.twinx()
ax2.step(support,y["Ozono"],color="green",label = "Ozono" )
ax2.set_ylabel("concentration [ppb]",color="red",fontsize=14)
ax.legend()
ax2.legend()

plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_val_s["Ozono"].values.reshape(len(y_val_s["Ozono"]),1), predictions[:,0], squared=False)**0.5
correlation_matrix = np.corrcoef(y_val_s["Ozono"].values, predictions[:,0].T)

correlation_xy = correlation_matrix[0,1]

r_squared = correlation_xy**2


supportografico = range(len(y_val_s) )
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
t = np.arange(0, predictions.size, 1)
ax2.plot(supportografico, y_val_s["Ozono"], 'b', label='Measured $\mathregular{O_3}$ concentration')
ax2.plot(supportografico, predictions[:,0], 'r', label='Predicted $\mathregular{O_3}$ concentration')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('$\mathregular{O_3}$ Prediction with all the eigth sensors 13/01/2021-29/04/2021')
ax2.set_xlabel('Validation Data')
ax2.set_ylabel('$\mathregular{O_3}$ concentration [\u03BCg/$\mathregular{m^3}$]')
ax2.text(0.02, 0.82, "$\mathregular{R^2}$ = " + str(round(r_squared,3)) + "  RMS  = "  + str(round(rms,3))+ " ppb", transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.legend()
plt.savefig('O3allRPredict.png', dpi=600, bbox_inches='tight')
plt.show()


fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico, y_val_s["Ozono"].values - predictions[:,0], 'y', label='Hourly difference between calculated-measured $\mathregular{O_3}$ concentration')
plt.plot(supportografico, [np.mean( y_val_s["Ozono"].values.reshape(len(y_val_s["Ozono"]),1) - predictions[:,0]) for i in range(len(y_val_s["Ozono"]) )], 'r', label='Mean value difference between calculated-measured $\mathregular{O_3}$ concentration')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('$\mathregular{O_3}$ Residues with all the eigth sensors 13/01/2021-29/04/2021')
plt.xlabel('Validation Data')
plt.ylabel('Difference between calculated and measured $\mathregular{O_3}$ (ppb)')
plt.legend()
plt.savefig('O3allR.png', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_val_s["Ossido di Carbonio"].values.reshape(len(y_val_s["Ossido di Carbonio"]),1), predictions[:,4], squared=False)**0.5
correlation_matrix = np.corrcoef(y_val_s["Ossido di Carbonio"].values, predictions[:,4].T)

correlation_xy = correlation_matrix[0,1]

r_squared = correlation_xy**2


supportografico = range(len(y_val_s) )
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
t = np.arange(0, predictions.size, 1)
ax2.plot(supportografico, y_val_s["Ossido di Carbonio"], 'y', label='Measured $\mathregular{CO}$ concentration')
ax2.plot(supportografico, predictions[:,4], 'r', label='Predicted $\mathregular{CO}$ concentration')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('$\mathregular{CO}$ Prediction with all the eigth sensors 13/01/2021-29/04/2021')
ax2.set_xlabel('Validation Data')
ax2.set_ylabel('$\mathregular{CO}$ concentration [mg/$\mathregular{m^3}$]')
ax2.text(0.60, 0.10, "$\mathregular{R^2}$ = " + str(round(r_squared,3)) + "  RMS  = "  + str(round(rms,3))+ " ppm", transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.legend()
plt.savefig('COallRPredict.png', dpi=600, bbox_inches='tight')
plt.show()


fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico, y_val_s["Ossido di Carbonio"].values - predictions[:,4], 'y', label='Hourly difference between calculated-measured $\mathregular{CO}$ concentration')
plt.plot(supportografico, [np.mean( y_val_s["Ossido di Carbonio"].values.reshape(len(y_val_s["Ossido di Carbonio"]),1) - predictions[:,4]) for i in range(len(y_val_s["Ossido di Carbonio"]) )], 'r', label='Mean value difference between calculated-measured $\mathregular{CO}$ concentration')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('$\mathregular{CO}$ Residues with all the eigth sensors 13/01/2021-29/04/2021')
plt.xlabel('Validation Data')
plt.ylabel('Difference between calculated and measured $\mathregular{CO}$ (ppm)')
plt.legend()
plt.savefig('COallR.png', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_val_s["PM10"].values.reshape(len(y_val_s["PM10"]),1), predictions[:,1], squared=False)**0.5
correlation_matrix = np.corrcoef(y_val_s["PM10"].values, predictions[:,1].T)

correlation_xy = correlation_matrix[0,1]

r_squared = correlation_xy**2


supportografico = range(len(y_val_s) )
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
t = np.arange(0, predictions.size, 1)
ax2.plot(supportografico, y_val_s["PM10"], 'y', label='Measured $\mathregular{PM10}$ concentration')
ax2.plot(supportografico, predictions[:,1], 'r', label='Predicted $\mathregular{PM10}$ concentration')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('$\mathregular{PM10}$ Prediction with all the eigth sensors 13/01/2021-29/04/2021')
ax2.set_xlabel('Validation Data')
ax2.set_ylabel('$\mathregular{PM10}$ concentration [\u03BCg/$\mathregular{m^3}$]')
ax2.text(0.02, 0.86, "$\mathregular{R^2}$ = " + str(round(r_squared,3)) + "  RMS  = "  + str(round(rms,3))+ " ppb", transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.legend()
plt.savefig('PM10allRPredict.png', dpi=600, bbox_inches='tight')
plt.show()


fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico, y_val_s["PM10"].values - predictions[:,1], 'y', label='Hourly difference between calculated-measured $\mathregular{PM10}$ concentration')
plt.plot(supportografico, [np.mean( y_val_s["PM10"].values.reshape(len(y_val_s["PM10"]),1) - predictions[:,1]) for i in range(len(y_val_s["PM10"]) )], 'r', label='Mean value difference between calculated-measured $\mathregular{PM10}$ concentration')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('$\mathregular{PM10}$ Residues with all the eigth sensors 13/01/2021-29/04/2021')
plt.xlabel('Validation Data')
plt.ylabel('Difference between calculated and measured $\mathregular{PM10}$ (ppb)')
plt.legend()
plt.savefig('PM10allR.png', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_val_s["Biossido di Azoto"].values.reshape(len(y_val_s["Biossido di Azoto"]),1), predictions[:,3], squared=False)**0.5
correlation_matrix = np.corrcoef(y_val_s["Biossido di Azoto"].values, predictions[:,3].T)

correlation_xy = correlation_matrix[0,1]

r_squared = correlation_xy**2


supportografico = range(len(y_val_s) )
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
t = np.arange(0, predictions.size, 1)
ax2.plot(supportografico, y_val_s["Biossido di Azoto"], 'y', label='Measured $\mathregular{NO_2}$ concentration')
ax2.plot(supportografico, predictions[:,3], 'r', label='Predicted $\mathregular{NO_2}$ concentration')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('$\mathregular{NO_2}$ Prediction with all the eigth sensors 13/01/2021-29/04/2021')
ax2.set_xlabel('Validation Data')
ax2.set_ylabel('$\mathregular{NO_2}$ concentration [mg/$\mathregular{m^3}$]')
ax2.text(0.02, 0.86, "$\mathregular{R^2}$ = " + str(round(r_squared,3)) + "  RMS  = "  + str(round(rms,3))+ " ppb", transform=ax2.transAxes, fontsize=14,verticalalignment='top', bbox=props)
ax2.legend()
plt.savefig('NO_2allRPredict.png', dpi=600, bbox_inches='tight')
plt.show()


fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico, y_val_s["Biossido di Azoto"].values - predictions[:,3], 'y', label='Hourly difference between calculated-measured $\mathregular{NO_2}$ concentration')
plt.plot(supportografico, [np.mean( y_val_s["Biossido di Azoto"].values.reshape(len(y_val_s["Biossido di Azoto"]),1) - predictions[:,3]) for i in range(len(y_val_s["PM10"]) )], 'r', label='Mean value difference between calculated-measured $\mathregular{NO_2}$ concentration')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('$\mathregular{NO_2}$ Residues with all the eigth sensors 13/01/2021-29/04/2021')
plt.xlabel('Validation Data')
plt.ylabel('Difference between calculated and measured $\mathregular{NO_2}$ (ppb)')
plt.legend()
plt.savefig('NO_2allR.png', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
corr = y.corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:
from pandas.plotting import table 

df9 = read_csv("29apr_1lug_appa1.csv",sep="\t",header=0)
df9 = df9.drop("Time",axis = 1)
df9 = df9.apply(pd.to_numeric, errors='coerce')
df9 = df9.dropna()
y2 = pd.DataFrame([df9.pop(i) for i in ["Ozono", "PM10","Biossido Zolfo", "Biossido di Azoto", "Ossido di Carbonio"]]).T



In [ ]:
corr = y.corr()


corr.style.background_gradient(cmap='coolwarm')




In [ ]:
corr2 = y2.corr()

corr2.style.background_gradient(cmap='coolwarm')

In [ ]:
df11 = read_csv("1lug_22set_appa1.csv",sep="\t",header=0)
df11 = df11.drop("Time",axis = 1)
df11 = df11.apply(pd.to_numeric, errors='coerce')
df11 = df11.dropna()
y4 = pd.DataFrame([df11.pop(i) for i in ["Ozono", "PM10","Biossido Zolfo", "Biossido di Azoto", "Ossido di Carbonio"]]).T

corr4 = y4.corr()

corr4.style.background_gradient(cmap='coolwarm')

In [ ]:


print(df8)

In [ ]:
prova15 = pd.merge(df8,df9,how = "outer")
prova16 = pd.merge(prova15,df11,how = "outer")
prova16

In [ ]:
supportografico = range(len(prova16))

fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico,prova16.R7, 'y', label='R7')
plt.plot(supportografico, prova16.R8, 'r', label='R8')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('Risposta Sensori R7 e R8 da 13/01/2021 a 22/09/2021')
plt.xlabel('Scala Temporale')
plt.ylabel('Risposta')
plt.legend()
plt.savefig('R7R8.png', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico,prova16.R1, 'y', label='R1')
plt.plot(supportografico, prova16.R2, 'r', label='R2')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('Risposta Sensori R1 e R2 da 13/01/2021 a 22/09/2021')
plt.xlabel('Scala Temporale')
plt.ylabel('Risposta')
plt.legend()
plt.savefig('R1R2.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico,prova16.R3, 'y', label='R3')
plt.plot(supportografico, prova16.R4, 'r', label='R4')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('Risposta Sensori R3 e R4 da 13/01/2021 a 22/09/2021')
plt.xlabel('Scala Temporale')
plt.ylabel('Risposta')
plt.legend()
plt.savefig('R3R4.png', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico,prova16.R5, 'y', label='R5')
plt.plot(supportografico, prova16.R6, 'r', label='R6')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('Risposta Sensori R5 e R6 da 13/01/2021 a 22/09/2021')
plt.xlabel('Scala Temporale')
plt.ylabel('Risposta')
plt.legend()
plt.savefig('R5R6.png', dpi=600, bbox_inches='tight')
plt.show()



In [ ]:
prova17 = prova16.loc[(prova16.R7 <  3 * 10**8)].loc[(prova16.R8 <  3 * 10**8)]

In [ ]:
supportografico = range(len(prova17))

fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico,prova17.R7, 'y', label='R7')
plt.plot(supportografico, prova17.R8, 'r', label='R8')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('Risposta Sensori R7 e R8 da 13/01/2021 a 22/09/2021')
plt.xlabel('Scala Temporale')
plt.ylabel('Risposta')
plt.legend()
plt.savefig('R7R8.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
yparz = pd.merge(y,y2,how = "outer")
ytot = pd.merge(yparz,y4,how = "outer")


corrtot = ytot.corr()

corrtot.style.background_gradient(cmap='coolwarm')

In [ ]:
ytot

In [ ]:
supportografico = range(len(X_train_scaled))

fig = plt.figure(figsize=(10,7))
t = np.arange(0, predictions.size, 1)
plt.plot(supportografico,X_train_scaled[:,6], 'y', label='R7')
plt.plot(supportografico, X_train_scaled[:,7], 'r', label='R8')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
plt.title('Risposta Sensori R7 e R8 da 13/01/2021 a 22/09/2021')
plt.xlabel('Scala Temporale')
plt.ylabel('Risposta')
plt.legend()
plt.savefig('R7R8.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
df8

In [ ]:
df8 = read_csv("13gen_29apr_appa1.csv",sep="\t",header=0)
df8 = df8.drop("Time",axis = 1)
df8 = df8.apply(pd.to_numeric, errors='coerce')
df8 = df8.dropna()

corrcomplete = df8.corr()

corrcomplete.style.background_gradient(cmap='coolwarm')

In [ ]:
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
#ax=ax2.twinx()
ax2.scatter(df8.Ozono[3:-10], df8.R1[3:-10], label='R1')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('R1 contro Ozono')
ax2.set_xlabel('Time')
ax2.set_ylabel('Signal')
ax2.legend()
plt.show()


In [ ]:
import scipy.stats
Valori = scipy.stats.linregress(df8["R1"],df8["Ozono"])
print(Valori)

In [ ]:
supportografico = range(len(df8) )
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
#ax=ax2.twinx()
t = np.arange(0, predictions.size, 1)
ax2.plot(supportografico[3:-10], df8.R3[3:-10], 'y', label='R3')
ax2.plot(supportografico[3:-10], df8.R4[3:-10], 'r', label='R4')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('R3-R4 contro Ozono')
ax2.set_xlabel('Time')
ax2.set_ylabel('Signal')
ax.set_ylabel('$O_3$ Concentration')
ax2.legend()
ax.legend()
plt.show()

fig = plt.figure(figsize=(10,7))

ax = fig.add_subplot()

ax.plot(supportografico[3:-10], df8.Ozono[3:-10], 'b', label='Ozono')
plt.show()


In [ ]:
supportografico = range(len(df8) )
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
#ax=ax2.twinx()
t = np.arange(0, predictions.size, 1)
ax2.plot(supportografico[3:-10], df8.R5[3:-10], 'y', label='R5')
ax2.plot(supportografico[3:-10], df8.R6[3:-10], 'r', label='R6')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('R5-R6 contro Ozono')
ax2.set_xlabel('Time')
ax2.set_ylabel('Signal')
ax.set_ylabel('$O_3$ Concentration')
ax2.legend()
ax.legend()
plt.show()

fig = plt.figure(figsize=(10,7))

ax = fig.add_subplot()

ax.plot(supportografico[3:-10], df8.Ozono[3:-10], 'b', label='Ozono')
plt.show()


In [ ]:
supportografico = range(len(df8) )
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
#ax=ax2.twinx()
t = np.arange(0, predictions.size, 1)
ax2.plot(supportografico[3:-10], df8.R7[3:-10], 'y', label='R7')
ax2.plot(supportografico[3:-10], df8.R8[3:-10], 'r', label='R8')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('R5-R6 contro Ozono')
ax2.set_xlabel('Time')
ax2.set_ylabel('Signal')
ax.set_ylabel('$O_3$ Concentration')
ax2.legend()
ax.legend()
plt.show()

fig = plt.figure(figsize=(10,7))

ax = fig.add_subplot()

ax.plot(supportografico[3:-10], df8.Ozono[3:-10], 'b', label='Ozono')
plt.show()


In [ ]:
df8 = read_csv("13gen_29apr_appa1.csv",sep="\t",header=0)
df8 = df8.dropna()
df8

In [ ]:
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
#ax=ax2.twinx()
t = np.arange(0, predictions.size, 1)
ax2.plot(df8.index[5:-10], df8.R3[5:-10], 'y', label='R3')
#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('R5-R6 contro Ozono')
ax2.set_xlabel('Time')
ax2.set_ylabel('Signal')
ax.set_ylabel('$O_3$ Concentration')
ax2.legend()
ax.legend()
plt.show()


In [ ]:
df8.loc[646:650].R4

In [ ]:
df8.loc[646:650].Time

In [ ]:
dfappa2 = read_csv("../APPA2/1lug_22set_appa.csv",sep="\t",header=0)
dfappa2 = dfappa2.drop("Time",axis = 1)
dfappa2 = dfappa2.apply(pd.to_numeric, errors='coerce')
dfappa2 = dfappa2.dropna()


In [ ]:
dfappa1 = read_csv("1lug_22set_appa1.csv",sep="\t",header=0)
dfappa1 = dfappa1.drop("Time",axis = 1)
dfappa1 = dfappa1.apply(pd.to_numeric, errors='coerce')
dfappa1 = dfappa1.dropna()


In [ ]:
dfappa12 = read_csv("29apr_1lug_appa1.csv",sep="\t",header=0)
dfappa12 = dfappa12.drop("Time",axis = 1)
dfappa12 = dfappa12.apply(pd.to_numeric, errors='coerce')
dfappa12 = dfappa12.dropna()

In [ ]:
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
#ax=ax2.twinx()
ax2.scatter(df8.index[700:-10], df8.T1[700:-10], label='APPA1 Inverno')
ax2.scatter(dfappa2.index[1000:-10], dfappa2.T4[1000:-10], label='APPA2')
ax2.scatter(dfappa1.index[3:-10], dfappa1.T5[3:-10], label='APPA1 Estate')
ax2.scatter(dfappa12.index[50:-50], dfappa12.T1[50:-50], label='APPA1 Primavera')

#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('ZnOR contro Ozono')
ax2.set_xlabel('Concentrazione di Ozono')
ax2.set_ylabel('Resistenza')
ax2.legend()
#plt.savefig('Inverno+APPA2.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
fig = plt.figure(figsize=(10,7))
ax2 = fig.add_subplot()
#ax=ax2.twinx()

ax2.scatter(df8.Ozono[700:-10], StandardScaler(df8.R1[700:-10]), label='APPA1 Inverno')
ax2.scatter(dfappa2.Ozono[1000:-10], StandardScaler(dfappa2.R4[1000:-10]), label='APPA2')
ax2.scatter(dfappa1.Ozono[3:-10], StandardScaler(dfappa1.R6[3:-10]), label='APPA1 Estate Canale 6')
ax2.scatter(dfappa1.Ozono[3:-10], StandardScaler(dfappa1.R5[3:-10]), label='APPA1 Estate Canale 5')

ax2.scatter(dfappa12.Ozono[50:-50], StandardScaler(dfappa12.R1[50:-50]), label='APPA1 Primavera')

#plt.plot(supportografico, predictions[:100,0], 'r', label='Prediction')
ax2.set_title('ZnOR contro Ozono')
ax2.set_xlabel('Concentrazione di Ozono')
ax2.set_ylabel('Resistenza')
ax2.legend()
#plt.savefig('Inverno+APPA2.png', dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
import scipy.stats
Valori = scipy.stats.linregress(dfappa2["R4"][1000:],dfappa2["Ozono"][1000:])
print(Valori)

In [ ]:
Valori2 = scipy.stats.linregress(df8["R1"][100:-100],df8["Ozono"][100:-100])
print(Valori2)